In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [297]:
df = pd.read_csv('train.csv')
df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location,Fault
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,...,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,...,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,...,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583 7.3783,0
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,...,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,"3.84,-70633",1
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,...,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917 7.0183,0


In [2]:
test_df = pd.read_csv('test.csv')
test_df.head()

C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Id,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,0,MP810M,NaN,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,...,25,No,No,2004,LEXUS,RX330,NO MISUSE,38.988440,-77.127668,"3.84,-71763"
1,1,MP2001,NaN,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,...,25,No,No,2001,MITSUBISHI,MIRAGE,NO MISUSE,39.094075,-77.205783,3.905 7.0733
2,2,MP6404,10125,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,...,35,No,No,2016,FRIEGHTLINER,AMBULANCE,NO MISUSE,39.047180,-77.052688,"3.41,-70283"
3,3,MP0405,10270,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,...,25,No,No,2004,FORD,CN,NaN,39.178007,-77.238475,3.7067 7.345
4,4,MP090D,10374,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,...,55,No,No,2008,UTIL,TL,UNKNOWN,39.056134,-77.154043,3.5143 7.5022


In [3]:
test_df['Id'].value_counts()

0        1
51496    1
51494    1
51493    1
51492    1
        ..
25744    1
25743    1
25742    1
25741    1
77234    1
Name: Id, Length: 77235, dtype: int64

In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Report Number                   51490 non-null  object 
 1   Local Case Number               45037 non-null  object 
 2   Agency Name                     51490 non-null  object 
 3   ACRS Report Type                51490 non-null  object 
 4   Crash Date/Time                 51490 non-null  object 
 5   Route Type                      46522 non-null  object 
 6   Road Name                       46988 non-null  object 
 7   Cross-Street Type               46471 non-null  object 
 8   Cross-Street Name               46968 non-null  object 
 9   Off-Road Description            6950 non-null   object 
 10  Municipality                    7567 non-null   object 
 11  Related Non-Motorist            3648 non-null   object 
 12  Collision Type                  

In [300]:
df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])
df['year'] = df['Crash Date/Time'].dt.year
df['month'] = df['Crash Date/Time'].dt.month
df['date'] = df['Crash Date/Time'].dt.day
df['day'] = df['Crash Date/Time'].dt.dayofweek
df['hour'] = df['Crash Date/Time'].dt.hour
df.drop('Crash Date/Time', axis = 1, inplace = True)

In [301]:
unique, counts = np.unique(df['Report Number'], return_counts = True)

In [302]:
counts.sum()

51490

In [303]:
df.drop('Report Number', axis = 1, inplace=True)
df.shape

(51490, 45)

In [304]:
df.drop('Local Case Number', axis = 1, inplace=True)

In [305]:
df.shape

(51490, 44)

In [306]:
df['Route Type'].value_counts()

Maryland (State)        22482
County                  16059
Municipality             2840
US (State)               2291
Interstate (State)       1145
Other Public Roadway      489
Ramp                      398
Government                390
Service Road              218
Unknown                   210
Name: Route Type, dtype: int64

In [307]:
df['Route Type'] = df['Route Type'].fillna(df['Route Type'].mode()[0])
df['Route Type'].describe()

count                51490
unique                  10
top       Maryland (State)
freq                 27450
Name: Route Type, dtype: object

In [308]:
df['Road Name'] = df['Road Name'].fillna(df['Road Name'].mode()[0])
df['Road Name'].describe()

count           51490
unique           2226
top       GEORGIA AVE
freq             7624
Name: Road Name, dtype: object

In [309]:
df['Cross-Street Type'] = df['Cross-Street Type'].fillna(df['Cross-Street Type'].mode()[0])
df['Cross-Street Type'].describe()

count      51490
unique        10
top       County
freq       29713
Name: Cross-Street Type, dtype: object

In [310]:
df['Cross-Street Name'] = df['Cross-Street Name'].fillna(df['Cross-Street Name'].mode()[0])
df['Cross-Street Name'].describe()

count           51490
unique           4525
top       GEORGIA AVE
freq             5138
Name: Cross-Street Name, dtype: object

In [311]:
df.drop('Off-Road Description', axis = 1, inplace = True)
df.shape

(51490, 43)

In [312]:
df['Municipality'] = df['Municipality'].fillna(df['Municipality'].mode()[0])
df['Municipality'].describe()

count         51490
unique           20
top       ROCKVILLE
freq          46629
Name: Municipality, dtype: object

In [313]:
df.drop('Related Non-Motorist', axis = 1, inplace = True)

In [314]:
df.shape

(51490, 42)

In [315]:
df['Collision Type'] = df['Collision Type'].fillna(df['Collision Type'].mode()[0])
df['Collision Type'].describe()

count                 51490
unique                   18
top       SAME DIR REAR END
freq                  17030
Name: Collision Type, dtype: object

In [316]:
df['Weather'] = df['Weather'].fillna(df['Weather'].mode()[0]) 
df['Weather'].describe()

count     51490
unique       12
top       CLEAR
freq      37143
Name: Weather, dtype: object

In [317]:
df['Surface Condition'] = df['Surface Condition'].fillna(df['Surface Condition'].mode()[0])
df['Surface Condition'].describe()

count     51490
unique       11
top         DRY
freq      40076
Name: Surface Condition, dtype: object

In [318]:
df['Light'].value_counts()

DAYLIGHT                    33617
DARK LIGHTS ON              11224
DARK NO LIGHTS               1716
DUSK                         1425
DAWN                         1222
DARK -- UNKNOWN LIGHTING      727
UNKNOWN                       480
OTHER                         375
Name: Light, dtype: int64

In [319]:
temp = df['Light'].loc[(df['hour'] >= 6) & (df['hour'] < 17)]
temp = temp.fillna('DAYLIGHT')
df.update(temp)

In [320]:
temp = df['Light'].loc[(df['hour'] >= 17) & (df['hour'] < 18)]
temp = temp.fillna('DAYLIGHT')
df.update(temp)

In [321]:
temp = df['Light'].loc[(df['hour'] >= 18) & (df['hour'] <= 23)]
temp = temp.fillna('DARK LIGHTS ON')
df.update(temp)

In [322]:
temp = df['Light'].loc[(df['hour'] >= 0) & (df['hour'] < 5)]
temp = temp.fillna('DARK LIGHTS ON')
df.update(temp)

In [323]:
temp = df['Light'].loc[(df['hour'] >= 5) & (df['hour'] < 6)]
temp = temp.fillna('DARK LIGHTS ON')
df.update(temp)

In [324]:
df['Light'].isnull().sum()

0

In [325]:
df['Light'].value_counts()

DAYLIGHT                    34114
DARK LIGHTS ON              11431
DARK NO LIGHTS               1716
DUSK                         1425
DAWN                         1222
DARK -- UNKNOWN LIGHTING      727
UNKNOWN                       480
OTHER                         375
Name: Light, dtype: int64

In [326]:
df['Traffic Control'] = df['Traffic Control'].fillna(df['Traffic Control'].mode()[0])
df['Traffic Control'].isnull().sum()

0

In [327]:
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].fillna(df['Driver Substance Abuse'].mode()[0])

In [328]:
df.drop('Non-Motorist Substance Abuse', axis = 1, inplace = True)

In [329]:
df.drop('Circumstance', axis = 1, inplace = True)

In [330]:
df['Drivers License State'] = df['Drivers License State'].fillna(df['Drivers License State'].mode()[0])
df['Drivers License State'].isnull().sum()

0

In [331]:
df['Vehicle Damage Extent'] = df['Vehicle Damage Extent'].fillna(df['Vehicle Damage Extent'].mode()[0])

In [332]:
df['Vehicle First Impact Location'] = df['Vehicle First Impact Location'].fillna(df['Vehicle First Impact Location'].mode()[0])

In [333]:
df['Vehicle Second Impact Location'] = df['Vehicle Second Impact Location'].fillna(df['Vehicle Second Impact Location'].mode()[0])

In [334]:
df['Vehicle Body Type'] = df['Vehicle Body Type'].fillna(df['Vehicle Body Type'].mode()[0])

In [335]:
df['Vehicle Movement'] = df['Vehicle Movement'].fillna(df['Vehicle Movement'].mode()[0])

In [336]:
df['Vehicle Continuing Dir'] = df['Vehicle Continuing Dir'].fillna(df['Vehicle Continuing Dir'].mode()[0])

In [337]:
df['Vehicle Going Dir'] = df['Vehicle Going Dir'].fillna(df['Vehicle Going Dir'].mode()[0])

In [338]:
df['Vehicle Make'] = df['Vehicle Make'].fillna(df['Vehicle Make'].mode()[0])

In [339]:
df['Vehicle Model'] = df['Vehicle Model'].fillna(df['Vehicle Model'].mode()[0])

In [340]:
df['Equipment Problems'] = df['Equipment Problems'].fillna(df['Equipment Problems'].mode()[0])

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Agency Name                     51490 non-null  object 
 1   ACRS Report Type                51490 non-null  object 
 2   Route Type                      51490 non-null  object 
 3   Road Name                       51490 non-null  object 
 4   Cross-Street Type               51490 non-null  object 
 5   Cross-Street Name               51490 non-null  object 
 6   Municipality                    51490 non-null  object 
 7   Collision Type                  51490 non-null  object 
 8   Weather                         51490 non-null  object 
 9   Surface Condition               51490 non-null  object 
 10  Light                           51490 non-null  object 
 11  Traffic Control                 51490 non-null  object 
 12  Driver Substance Abuse          

## Converting categorical data to numeric data

In [342]:
col_ls = df._get_numeric_data().columns
categorical_ls = list(set(df.columns) - set(col_ls))

le = LabelEncoder()

for col_name in categorical_ls:
    df[col_name] = le.fit_transform(df[col_name])

In [343]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Agency Name                     51490 non-null  int32  
 1   ACRS Report Type                51490 non-null  int32  
 2   Route Type                      51490 non-null  int32  
 3   Road Name                       51490 non-null  int32  
 4   Cross-Street Type               51490 non-null  int32  
 5   Cross-Street Name               51490 non-null  int32  
 6   Municipality                    51490 non-null  int32  
 7   Collision Type                  51490 non-null  int32  
 8   Weather                         51490 non-null  int32  
 9   Surface Condition               51490 non-null  int32  
 10  Light                           51490 non-null  int32  
 11  Traffic Control                 51490 non-null  int32  
 12  Driver Substance Abuse          

## Normalizing the columns

In [346]:
mms = MinMaxScaler()
cols = df.columns
vals = df.values
vals_scaled = mms.fit_transform(vals)
df = pd.DataFrame(vals_scaled, columns=cols)

In [353]:
X,y = df.drop('Fault',axis=1), df['Fault']

In [ ]:
# feat_sel = ExtraTreesClassifier()
# feat_sel.fit(X,y)
# feat_imp = pd.Series(feat_sel.feature_importances_,index = X.columns)
# feat_imp.nlargest(15).plot(kind='barh')
# plt.show()